In [1]:
# Importacion de librerias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import KFold

In [2]:
def readData(rutaArchivo, formato='csv'):
    if formato == 'csv':
        data = pd.read_csv(rutaArchivo)
    elif formato == 'arff':
        data = arff.loadarff(rutaArchivo)
        data = pd.DataFrame(data[0])
    return data

def getNombreClases(data, nombreClase, formato='csv'):
    nombreClases = data[nombreClase].unique()
    if formato == 'arff':
        nombreClases = [clase.decode('utf-8') if isinstance(clase, bytes) else clase for clase in nombreClases]
    return nombreClases

In [ ]:
def getFrequencyCounts(data, clases):
    frequency_counts = {}

    for x_val, clase_val in zip(data.iloc[:, 0], data.iloc[:, -1]):
        if x_val not in frequency_counts:
            frequency_counts[x_val] = {cl: 0 for cl in clases}

        frequency_counts[x_val][clase_val] += 1

    frequency_counts = {
        x: {c: frequency_counts[x][c] for c in sorted(frequency_counts[x])}
        for x in sorted(frequency_counts)
    }

    return frequency_counts

In [ ]:
def gini_columna(counts):
    """
    Calcula la impureza de Gini para cada hoja.
    Parámetros:
    @data: DataFrame de pandas que contiene la frecuencia de cada clase en una hoja.

    Retorna:
    @gini: Valor de la impureza de Gini para la hoja.
    """
    total = counts.sum()
    p = counts / total

    return 1 - np.sum(p ** 2)

def getImpurezaGini(data):
    """
    Calcula la impureza de Gini para un conjunto de datos dado.
    Parámetros:
    @data: DataFrame de pandas que contiene la frecuencia de cada clase.

    Retorna:
    @sumaPonderadaGini: Valor de la impureza de Gini para el atributo.
    """

    gini_por_hoja = data.apply(gini_columna, axis=0)

    # --- Pesos (proporción de elementos por hoja) ---
    tam = data.sum(axis=0)   # total por hoja
    total = tam.sum()

    pesos = tam / total

    # Calculo de la Gini ponderada
    gini_ponderada = (pesos * gini_por_hoja).sum()

    return gini_por_hoja, gini_ponderada

In [ ]:
# Main
data = readData('iris.csv', formato='csv')
c = 1
print(data.head())

print("--------------------")
print(data.columns[c])

print("********************")
print(data.iloc[:, c].dtype)

print("********************")
print(data.iloc[:, c])

print("--------------------")

clasesNombre = getNombreClases(data, "Species", formato='csv')
print("Clases: ", clasesNombre)




# gini = getImpurezaGini(data.iloc[:, c], clasesNombre)
# print(gini)

   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa
3   4            4.6           3.1            1.5           0.2  Iris-setosa
4   5            5.0           3.6            1.4           0.2  Iris-setosa
--------------------
SepalLengthCm
********************
float64
********************
0      5.1
1      4.9
2      4.7
3      4.6
4      5.0
      ... 
145    6.7
146    6.3
147    6.5
148    6.2
149    5.9
Name: SepalLengthCm, Length: 150, dtype: float64
--------------------
Clases:  ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [ ]:
def getMeans(data, atributo, clasesNombre):
    if np.issubdtype(data[atributo].dtype, np.number):
        print(f"El atributo '{atributo}' es numérico.")
        # Dividir el atributo en dos partes usando la media
        umbral = data[atributo].mean()
        print(f"Umbral (media) para '{atributo}': {umbral}")
        
        menor_igual = data[data[atributo] <= umbral]
        mayor = data[data[atributo] > umbral]
        
        # Crear un nuevo DataFrame con las frecuencias de clases para cada parte
        frecuencias_menor_igual = getFrequencyCounts(menor_igual, clasesNombre)
        frecuencias_mayor = getFrequencyCounts(mayor, clasesNombre)
        
        nuevo_df = pd.DataFrame({
            f'<= {umbral}': pd.Series(frecuencias_menor_igual).apply(lambda x: x if isinstance(x, dict) else {}),
            f'> {umbral}': pd.Series(frecuencias_mayor).apply(lambda x: x if isinstance(x, dict) else {})
        }).fillna(0).astype(int)
        
        print("DF Frecuencias:\n", nuevo_df)
        
        giniHojas, gini = getImpurezaGini(nuevo_df)
        print("--------------------")
        
        return nuevo_df, giniHojas, gini

In [ ]:
# Ejemplo


gustoCine = {
    "Id": [1, 2, 3, 4, 5, 6, 7],
    "Palomitas": ["Sí", "Sí", "No", "No", "Sí", "Sí", "No"],
    "Malteada": ["Sí", "No", "Sí", "Sí", "Sí", "No", "No"],
    "Edad": [7, 12, 18, 35, 38, 50, 83],
    "Cine": ["No", "No", "Sí", "Sí", "Sí", "No", "No"]   
}

data = pd.DataFrame(gustoCine)
c = 1
# print(data.head())

# print("--------------------")
# print(data.columns[c])

# print("********************")
# print(data.iloc[:, c].dtype)

# print("********************")
# print(data.iloc[:, c])

# print("--------------------")

clasesNombre = getNombreClases(data, "Cine", formato='csv')
print("Clases: ", clasesNombre)

data.drop(columns=["Id"], inplace=True)

nAtributos = data.columns[:-1]
clase = data.columns[-1]

# Creacion de nuevos dataframes por atributo, con la columna de clase
for atributo in nAtributos:
    # print("Atributo: ", nombre)
    # print("Datos atributo: ", data[nombre])
    frecuencias = {}
    df_fq = pd.DataFrame()

    nuevo_df = pd.DataFrame({
        atributo: data[atributo],
        "Clase": data[clase]
    })

    print("--------------------")
    print("Atributo: ", atributo)
    # print("Nuevo DF:\n", nuevo_df)

    # Detectando si es un dato categórico o numérico
    if pd.api.types.is_numeric_dtype(nuevo_df[atributo]):
        print(f"El atributo '{atributo}' es numérico.")
        
        
    else:
        print(f"El atributo '{atributo}' es categórico.")
        # print("Nuevo DF:\n", nuevo_df)
        frecuencias = getFrequencyCounts(nuevo_df, clasesNombre)
        df_fq = pd.DataFrame(frecuencias)
        # print("Frecuencias: ", frecuencias)
        print("DF Frecuencias:\n", df_fq)
        giniHojas, gini = getImpurezaGini(df_fq)
        print("--------------------")
        print("Gini hojas: ", giniHojas)
        print("Impureza Gini: ", gini)

    
    
    # print("Impureza Gini: ", gini)

Clases:  ['No' 'Sí']
--------------------
Atributo:  Palomitas
El atributo 'Palomitas' es categórico.
DF Frecuencias:
     No  Sí
No   1   3
Sí   2   1
Gini por hoja:  No    0.444444
Sí    0.375000
dtype: float64
--------------------
Gini hojas:  No    0.444444
Sí    0.375000
dtype: float64
Impureza Gini:  0.40476190476190477
--------------------
Atributo:  Malteada
El atributo 'Malteada' es categórico.
DF Frecuencias:
     No  Sí
No   3   1
Sí   0   3
Gini por hoja:  No    0.000
Sí    0.375
dtype: float64
--------------------
Gini hojas:  No    0.000
Sí    0.375
dtype: float64
Impureza Gini:  0.21428571428571427
--------------------
Atributo:  Edad
El atributo 'Edad' es numérico.
